# Rending a Chord Diagram in a Jupyter Notebook using D3.js

Based on:

https://bl.ocks.org/mbostock/4062006#index.html

and

https://github.com/cmoscardi/embedded_d3_example

In [11]:
import pandas as pd
import numpy as np
import json
from IPython.display import Javascript
from IPython.core.display import HTML
HTML("""
<style>
.chord path {
  fill-opacity: .67;
  stroke: #000;
  stroke-width: .5px;
}</style>
""")

In [12]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [21]:
matrix = pd.DataFrame( [  [ 10,  1, 2, 3],
                          [ 1, 10, 1, 1],
                          [ 2, 1, 10, 1],
                          [ 3,   1,  1, 10]])
matrix

,0,1,2,3
0,10,1,2,3
1,1,10,1,1
2,2,1,10,1
3,3,1,1,10


In [22]:
jsmatrix = json.dumps(matrix.as_matrix().tolist())
Javascript("""
           window.matrix={};
           """.format(jsmatrix))

<IPython.core.display.Javascript object>

In [23]:
%%javascript
require(['d3'], function(d3) {
            //a weird idempotency thing
            $("#chart1").remove();
            //create canvas
            element.append("<div id='chart1'></div>");
            $("#chart1").width("960px");
            $("#chart1").height("600px");

            var chord = d3.layout.chord()
                .padding(.05)
                .sortSubgroups(d3.descending)
                .matrix(matrix);

            var width = 500,
                height = 500,
                innerRadius = Math.min(width, height) * .41,
                outerRadius = innerRadius * 1.1;

            var fill = d3.scale.ordinal()
                .domain(d3.range(4))
                .range(["#000000", "#FFDD89", "#957244", "#F26223"]);

            var svg = d3.select("#chart1").append("svg")
                .attr("width", width)
                .attr("height", height)
                .append("g")
                .attr("transform", "translate(" + width / 2 + "," + height / 2 + ")");

            svg.append("g").selectAll("path")
                .data(chord.groups)
                .enter().append("path")
                .style("fill", function(d) {
                    return fill(d.index);
                })
                .style("stroke", function(d) {
                    return fill(d.index);
                })
                .attr("d", d3.svg.arc().innerRadius(innerRadius).outerRadius(outerRadius))
                .on("mouseover", fade(.1))
                .on("mouseout", fade(1));

            var ticks = svg.append("g").selectAll("g")
                .data(chord.groups)
                .enter().append("g").selectAll("g")
                .data(groupTicks)
                .enter().append("g")
                .attr("transform", function(d) {
                    return "rotate(" + (d.angle * 180 / Math.PI - 90) + ")" + "translate(" + outerRadius + ",0)";
                });

            ticks.append("line")
                .attr("x1", 1)
                .attr("y1", 0)
                .attr("x2", 5)
                .attr("y2", 0)
                .style("stroke", "#000");

            ticks.append("text")
                .attr("x", 8)
                .attr("dy", ".35em")
                .attr("transform", function(d) {
                    return d.angle > Math.PI ? "rotate(180)translate(-16)" : null;
                })
                .style("text-anchor", function(d) {
                    return d.angle > Math.PI ? "end" : null;
                })
                .text(function(d) {
                    return d.label;
                });

            svg.append("g")
                .attr("class", "chord")
                .selectAll("path")
                .data(chord.chords)
                .enter().append("path")
                .attr("d", d3.svg.chord().radius(innerRadius))
                .style("fill", function(d) {
                    return fill(d.target.index);
                })
                .style("opacity", 1);

            // Returns an array of tick angles and labels, given a group.
            function groupTicks(d) {
                var k = (d.endAngle - d.startAngle) / d.value;
                return d3.range(0, d.value, 1).map(function(v, i) {
                    return {
                        angle: v * k + d.startAngle,
                        label: i % 5 ? null : v
                    };
                });
            }

            // Returns an event handler for fading a given chord group.
            function fade(opacity) {
                return function(g, i) {
                    svg.selectAll(".chord path")
                        .filter(function(d) {
                            return d.source.index != i && d.target.index != i;
                        })
                        .transition()
                        .style("opacity", opacity);
                };
            }
});

<IPython.core.display.Javascript object>